In [1]:
import spacy
import numpy as np
import pandas as pd
from rasa_nlu.model import Interpreter

In [2]:
nlp = spacy.load('en')

In [3]:
interpreter = Interpreter.load('./models/faq_chatbot_nlu/default/faq_chatbot_nlu')

C:\Users\Utkarsh Sah\Anaconda3\lib\site-packages\rasa_nlu\extractors\entity_synonyms.py:85: UserWarning: Failed to load synonyms file from './models/faq_chatbot_nlu/default/faq_chatbot_nlu\entity_synonyms.json'
  "".format(entity_synonyms_file))


In [4]:
def run_nlu(query):
    global interpreter
    return interpreter.parse(query)

In [40]:
def bot():
    print('********** Chatbot Loaded! Start typing your queries here **********')
    print('\n')
    query = ''
    while(True):
        query = input()
        query = query.lower()
        if (query=='exit'):
            break
        parsed_query = run_nlu(query)
        response = route(parsed_query)
        print('A: ',response)
        print('\n')

In [76]:
def route(parsed_query):
    response = 'Kindly rephrase your words'
    intent = parsed_query['intent']['name']
    print('( Intent:',intent,')')
    if (intent == 'greet' or intent == 'gratitude' or intent == 'goodbye'):
        response = general_conversation(intent)
    if (intent == 'requirement'):
        response = requirement(parsed_query)
    if (intent == 'extent'):
        response = single_entity_routing(parsed_query, intent ,'body')
    if (intent == 'definition'):
        response = single_entity_routing(parsed_query, intent ,'subject')
    if (intent == 'duration'):
        response = single_entity_routing(parsed_query, intent ,'subject')
    if (intent == 'benefits'):
        response = single_entity_routing(parsed_query, intent ,'subject')
    if (intent == 'legislation'):
        response = single_entity_routing(parsed_query, intent ,'field')
    if (intent == 'purpose'):
        response = single_entity_routing(parsed_query, intent ,'subject')
    if (intent == 'conditions'):
        response = single_entity_routing(parsed_query, intent ,'subject')
    if (intent == 'validity'):
        response = single_entity_routing(parsed_query, intent ,'country')
    if (intent == 'procedure'):
        response = single_entity_routing(parsed_query, intent ,'subject')
    if (intent == 'services'):
        response = single_entity_routing(parsed_query, intent ,'body')
    return response

In [77]:
def single_entity_routing(parsed_query, intent, req_entity):
    global nlp
    print(parsed_query)
    response = 'Kindly rephrase your words'
    path = 'routings/'+intent+'.xlsx'
    routings = pd.read_excel(path)
    entities = parsed_query['entities']
    if (len(entities) == 0):
        return response
    body = []
    for entity in entities:
        if (entity['entity'] == req_entity):
            body.append(entity['value'])
    if (len(body) == 0):
        return response
    body = body[0]
    
    max_similarity = None
    max_similar_row = None
    for item in routings[req_entity]:
        w1 = nlp(body)
        w2 = nlp(item)
        similarity = w1.similarity(w2)
        if (max_similarity == None):
            max_similarity = similarity
            max_similar_row = routings[routings[req_entity] == item]
        if (max_similarity < similarity):
            max_similarity = similarity
            max_similar_row = routings[routings[req_entity] == item]            
    if (len(max_similar_row) == 0):
        return response
    for item in max_similar_row['response']:
        response = item
    return response

In [78]:
def requirement(parsed_query):
    global nlp
    response = 'Kindly rephrase your words'
    requirement_routings = pd.read_excel('routings/requirement.xlsx')
    entities = parsed_query['entities']
    
    if (len(entities) == 0):
        return response
    
    status = []
    for entity in entities:
        if (entity['entity'] == 'status'):
            status.append(entity['value'])
    if (len(status) == 0):
        print('Please enter a status card that you hold')
        flag = 0
        while(flag == 0):
            status_query = input().lower()
            status_query_entities = run_nlu(status_query)['entities']
            if (len(status_query_entities) == 0):
                print('Kindly rephrase your words and enter a valid status')
            else:
                status_detected = status_query_entities[0]['value']
                flag = 1   
        status.append(status_detected)

    status = status[0]
    data = requirement_routings[requirement_routings['status'].isnull() == False]   
    required_row = data[data['status'] == status]
    if (len(required_row) == 0):
        return 'Sorry! No such information found.'
    response = required_row['response'][0]
    return response

In [ ]:
def general_conversation(intent):
    if (intent == 'greet'):
        return "Hi! I'm the Invest India Visa Chatbot"
    if (intent == 'gratitude'):
        return "It was a pleasure helping you"
    if (intent == 'goodbye'):
        return "Goodbye! Would love to hear from you again"

In [ ]:
bot()

********** Chatbot Loaded! Start typing your queries here **********


what are the services offered at embassies
( Intent: services )
{'intent': {'name': 'services', 'confidence': 0.57291154154843449}, 'entities': [], 'intent_ranking': [{'name': 'services', 'confidence': 0.57291154154843449}, {'name': 'address', 'confidence': 0.062128085414184631}, {'name': 'contact', 'confidence': 0.042269527213155564}, {'name': 'benefits', 'confidence': 0.042010915054758895}, {'name': 'extent', 'confidence': 0.040426126967109871}, {'name': 'legislation', 'confidence': 0.034832872960216393}, {'name': 'conditions', 'confidence': 0.030585435501493743}, {'name': 'types', 'confidence': 0.026148823400381903}, {'name': 'email', 'confidence': 0.025629301864322846}, {'name': 'procedure', 'confidence': 0.023415184852953162}], 'text': 'what are the services offered at embassies'}
A:  Kindly rephrase your words


services at embassies
( Intent: services )
{'intent': {'name': 'services', 'confidence': 0.95466784